In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.rUvb5rGnyJ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.fjVVx889BJ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.HrOrTcc645/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [2]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from tqdm import tqdm
import pandas as pd

options = Options()
options.add_argument("--no-sandbox")

options.headless = True



<ipython-input-2-5bdcb4c4125e>:11: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [3]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)
# ...
# driver.quit()
url = "https://www.bokuyaba.online/manga/the-dangers-in-my-heart-chapter-1/"
driver.get(url)
time.sleep(2)

In [10]:
path = '.wp-manga-chapter-img'
elems = driver.find_elements(By.CSS_SELECTOR, path)

In [11]:
srcs = []
for i in range(len(elems)):
    srcs.append(elems[i].get_attribute("src"))

In [12]:
from tqdm.notebook import tqdm
for i, src in tqdm(enumerate(srcs)):
    save_path = '/content/drive/MyDrive/for_work/ControlNet-v1-1-nightly/sample/' + str(i) + '.jpg'
    !wget -nv -O {save_path} {src}
    # print(src)


0it [00:00, ?it/s]

2023-06-13 01:11:07 URL:https://img.spoilerhat.com/img/?url=https://zjcdn.mangafox.me/store/manga/27320/01-001.0/compressed/d20190212_200638_1332.jpg [133137/133137] -> "/content/drive/MyDrive/for_work/ControlNet-v1-1-nightly/sample/0.jpg" [1]
2023-06-13 01:11:07 URL:https://img.spoilerhat.com/img/?url=https://zjcdn.mangafox.me/store/manga/27320/01-001.0/compressed/d20190212_200638_1333.jpg [26149/26149] -> "/content/drive/MyDrive/for_work/ControlNet-v1-1-nightly/sample/1.jpg" [1]
2023-06-13 01:11:07 URL:https://img.spoilerhat.com/img/?url=https://zjcdn.mangafox.me/store/manga/27320/01-001.0/compressed/d20190212_200638_1334.jpg [444310/444310] -> "/content/drive/MyDrive/for_work/ControlNet-v1-1-nightly/sample/2.jpg" [1]
2023-06-13 01:11:07 URL:https://img.spoilerhat.com/img/?url=https://zjcdn.mangafox.me/store/manga/27320/01-001.0/compressed/d20190212_200638_1335.jpg [18631/18631] -> "/content/drive/MyDrive/for_work/ControlNet-v1-1-nightly/sample/3.jpg" [1]
2023-06-13 01:11:08 URL:http

In [8]:
driver.quit()